<a href="https://colab.research.google.com/github/jessenb16/Data-Science-Bootcamp-Homework/blob/main/Homework_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd

df = pd.read_csv('employee.csv')


In [38]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", 101)

In [23]:
df.isnull().sum()


id                               0
timestamp                        0
country                          0
employment_status                0
job_title                        0
job_years                        0
is_manager                       0
hours_per_week                   0
telecommute_days_per_week        0
education                        0
is_education_computer_related    0
certifications                   0
salary                           0
dtype: int64

In [17]:
df.loc[df['hours_per_week'].isna(), 'hours_per_week'] = df['hours_per_week'].median()
df.loc[df['telecommute_days_per_week'].isna(), 'telecommute_days_per_week'] = df['telecommute_days_per_week'].median()
df = df.dropna()

In [28]:
df=df.drop(columns=['id', 'timestamp', 'country' ])

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4261 entries, 0 to 4276
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   employment_status              4261 non-null   object 
 1   job_title                      4261 non-null   object 
 2   job_years                      4261 non-null   float64
 3   is_manager                     4261 non-null   object 
 4   hours_per_week                 4261 non-null   float64
 5   telecommute_days_per_week      4261 non-null   float64
 6   education                      4261 non-null   object 
 7   is_education_computer_related  4261 non-null   object 
 8   certifications                 4261 non-null   object 
 9   salary                         4261 non-null   float64
dtypes: float64(4), object(6)
memory usage: 366.2+ KB


In [45]:
dft = df.copy()
cat_cols = [c for c in dft.columns if dft[c].dtype == 'object'
            and c not in ['is_manager', 'certifications']]
cat_data = dft[cat_cols]
cat_cols

['employment_status',
 'job_title',
 'education',
 'is_education_computer_related']

In [46]:
bin_cols = ['is_manager', 'certifications']
for c in bin_cols:
    dft[c] = dft[c].replace(to_replace=['Yes'], value=1)
    dft[c] = dft[c].replace(to_replace=['No'], value=0)

In [47]:
final_dft = pd.get_dummies(dft, columns=cat_cols, drop_first=True, dtype=int)
final_dft.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4261 entries, 0 to 4276
Data columns (total 25 columns):
 #   Column                                                        Non-Null Count  Dtype  
---  ------                                                        --------------  -----  
 0   job_years                                                     4261 non-null   float64
 1   is_manager                                                    4261 non-null   int64  
 2   hours_per_week                                                4261 non-null   float64
 3   telecommute_days_per_week                                     4261 non-null   float64
 4   certifications                                                4261 non-null   int64  
 5   salary                                                        4261 non-null   float64
 6   employment_status_Independent or freelancer or company owner  4261 non-null   int64  
 7   employment_status_Part time                                   4261 non-nul

In [48]:
y = final_dft['salary']
X = final_dft.drop(columns=['salary'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [54]:
num_cols = ['job_years', 'hours_per_week', 'telecommute_days_per_week']
scaler = StandardScaler()
scaler.fit(X_train[num_cols])
X_train[num_cols] = scaler.transform(X_train[num_cols])

In [60]:
reg = LinearRegression()
reg.fit(X_train, y_train)
LinearRegression()

LinearRegression()

In [63]:
reg.coef_

array([  180.49376824,    50.039116  ,   191.55123741,   222.30363   ,
          82.13963572,   -41.39481292,   147.98521623, -2209.39268975,
        1050.92710458,   830.72459961,   767.41031478,   500.13073073,
         975.87581766,   805.65290748,    46.11759783,   915.074053  ,
         693.30554827,  3518.66940974,   372.14444707,   -22.29039115,
         229.39565408,   222.49462987,  -317.56469605,  -167.80424186])

In [64]:
reg.intercept_

6149.859917628925

In [65]:
mean_squared_error(y_train,reg.predict(X_train))/np.mean(y_train)

209.58418281685852

In [66]:
y_pred = reg.predict(X_test)

In [67]:
mse=mean_squared_error(y_pred, y_test)/np.mean(y_test)
print("Mean squared error", mse)

Mean squared error 11328.900069595293


In [68]:
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mae)

Mean Absolute Error: 8863.553389142065
